In [1]:
# spaCy based imports
import spacy
nlp = spacy.load('fr_core_news_sm')

In [2]:
# Usual imports 
import pandas as pd
from tqdm import tqdm, tqdm_notebook

# NLP imports
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
import stop_words
import re, sys
from gensim.models import phrases, Phrases

In [3]:
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""
pd.set_option('display.max_colwidth', -1)

import nltk
nltk.download('punkt')

In [4]:
tqdm.pandas()
tqdm_notebook()

0it [00:00, ?it/s]

In [5]:
# Raw data imports
answers = pd.read_csv('raw/answers.csv', encoding="utf-8")
answers['theme'].unique()

array(['Démocratie et citoyenneté', 'Impôts et dépenses publiques',
       "Organisation de l'Etat", 'Contribution libre',
       'Transition écologique'], dtype=object)

In [6]:
sw = stop_words.get_stop_words(language='fr')

In [7]:
answers['tokens'] = answers.answer.map(lambda text: [w for w in text.replace('.', '').replace(',', '').replace('!', '').split() if w])
answers['tokens'] = answers.answer.map(lambda text: word_tokenize(text))
answers['tokens_clean'] =  answers.tokens.map(
                                    lambda tok: [t.lower() for t in re.split(" ", re.sub(r"(\W+|_|\d+)", " ", " ".join(tok))) \
                                                 if t.lower() not in sw and len(t)>1]
                        )
answers['tokens_clean_joined'] =  answers.tokens_clean.map(lambda tok: ' '.join(tok))

In [9]:
lemmatizer = FrenchLefffLemmatizer()

def lemmatize_answer(answer):
    lemmatized_answer = ''
    nlp_answer = nlp(answer)
    for token in nlp_answer:
        lemmatized_answer += lemmatizer.lemmatize(str(token)) + ' '
    return lemmatized_answer

tqdm.pandas()
answers['lemmatized_answer'] = answers['tokens_clean_joined'].progress_apply(lemmatize_answer)

100%|██████████| 19183/19183 [03:01<00:00, 105.44it/s]


In [11]:
answers.to_csv('preprocessed/answers_lemmatized.csv', sep='\t', encoding='utf-8')